<a href="https://colab.research.google.com/github/sandeep-006/Customer_lifetime_value_pred/blob/main/Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Validating data


In [6]:
import pandas as pd
import numpy as np
import re


In [7]:

# 1. Load the dataset
df = pd.read_csv('dataset_with_impurities.csv')
df


,transaction_id,transaction_date,store_id,store_name,store_city,store_region,customer_id,first_name,email,customer_phone,...,quantity,promotion_id,promotion_name,discount_percentage,total_price,discount_amount,final_amount,month,day_of_week,customer_tenure_days
0,DUPLICATE_ID,2024-07-24 09:59:14,S09,HCL Mart Chandigarh,Chandigarh,North,C163,Ravi,ravi50@gmail.com,9134584158,...,2,NaN,NaN,0,2998,0.0,2998.0,7,2,1477
1,T3945,2023-01-16 02:43:57,S01,HCL Mart Delhi,Delhi,North,C023,John,john44@gmail.com,8404921393,...,5,NaN,NaN,0,7500,0.0,7500.0,1,0,831
2,T3025,2023-12-18 15:14:03,S02,HCL Mart Bhubaneswar,Bhubaneswar,East,C039,Anil,anil92@gmail.com,8542278768,...,2,NaN,NaN,0,1598,0.0,1598.0,12,0,287
3,T1264,2024-11-30 20:40:00,S10,HCL Mart Pune,Pune,West,C183,Vikram,vikram67@gmail.com,7784753826,...,1,PR02,Apparel Fest,20,1999,399.8,1599.2,11,5,282
4,T5351,2024-11-13 13:28:03,S09,HCL Mart Ahmedabad,Ahmedabad,West,C009,Anil,anil66@gmail.com,8431845474,...,1,NaN,NaN,0,299,0.0,299.0,11,2,411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,T4773,2023-02-18 13:21:25,S16,HCL Mart Ahmedabad,Ahmedabad,West,C115,Priya,priya10@gmail.com,9704539471,...,3,NaN,NaN,0,3600,0.0,3600.0,2,5,265
6996,T6192,2023-11-27 18:44:37,S19,HCL Mart Hyderabad,Hyderabad,South,C140,Anil,anil44@gmail.com,9835664178,...,2,PR06,Back to School,10,900,90.0,810.0,11,0,724
6997,T6227,2024-01-19 17:17:02,S10,HCL Mart Bangalore,Bangalore,South,C123,Ravi,ravi81@gmail.com,7478122447,...,2,NaN,NaN,0,700,0.0,700.0,1,4,99
6998,T6391,2024-05-22 00:00:00,S18,HCL Mart Delhi,Delhi,North,C173,Priya,priya52@gmail.com,8390604558,...,5,PR14,Mid-Week Treats,8,6000,480.0,5520.0,5,2,104


In [8]:

# 2. Define Validation Rules
def get_validation_mask(df):
    # Rule A: Check for Missing Critical Values
    mask_missing = df[['transaction_id', 'customer_id', 'product_id', 'transaction_date']].isnull().any(axis=1)

    # Rule B: Check for Calculation Errors (Total Price should = Price * Qty)
    # We use round() to avoid small floating point discrepancies
    expected_total = df['unit_price'] * df['quantity']
    mask_calc = (df['total_price'] - expected_total).abs() > 0.01

    # Rule C: Check for Negative Values
    mask_negative = (df['quantity'] <= 0) | (df['unit_price'] <= 0) | (df['total_price'] <= 0)

    # Rule D: Check for Temporal Errors (Transaction before Customer Joined)
    t_date = pd.to_datetime(df['transaction_date'], errors='coerce')
    s_date = pd.to_datetime(df['customer_since'], errors='coerce')
    mask_temporal = (t_date < s_date) | t_date.isnull()

    # Rule E: Check for Email Format
    # Simply checking for existence of '@' as a basic validator
    mask_email = ~df['email'].str.contains('@', na=False)

    # Rule F: Check for Location Mismatch (Specific impurity we added)
    # Mumbai should be in 'West'
    mask_location = (df['store_city'] == 'Mumbai') & (df['store_region'] != 'West')

    # Combine all rules: If any rule is True, the row is "Impure"
    is_impure = mask_missing | mask_calc | mask_negative | mask_temporal | mask_email | mask_location
    return is_impure


In [9]:

# 3. Apply Validation
impure_mask = get_validation_mask(df)


In [10]:

# 4. Segregate into Two Dataframes
valid_df = df[~impure_mask].copy()
invalid_df = df[impure_mask].copy()


In [11]:

# 5. Save to separate CSV files
valid_df.to_csv('validated_data.csv', index=False)
invalid_df.to_csv('invalid_data.csv', index=False)


In [12]:

# 6. Final Report
print(f"Validation Complete!")
print(f"-------------------------------")
print(f"Total Records Processed: {len(df)}")
print(f"Clean Records Saved:     {len(valid_df)} (to 'validated_data.csv')")
print(f"Error Records Saved:     {len(invalid_df)} (to 'invalid_data.csv')")

Validation Complete!
-------------------------------
Total Records Processed: 7000
Clean Records Saved:     6575 (to 'validated_data.csv')
Error Records Saved:     425 (to 'invalid_data.csv')


In [13]:
idf=pd.read_csv('invalid_data.csv')
idf

,transaction_id,transaction_date,store_id,store_name,store_city,store_region,customer_id,first_name,email,customer_phone,...,quantity,promotion_id,promotion_name,discount_percentage,total_price,discount_amount,final_amount,month,day_of_week,customer_tenure_days
0,T3302,NaN,S11,HCL Mart Mumbai,Mumbai,West,C088,Vikram,vikram31@gmail.com,7895949284,...,1,NaN,NaN,0,299,0.0,299.0,2,6,492
1,T5161,2023-01-01 10:00:00,S16,HCL Mart Bangalore,Bangalore,South,C150,Ravi,ravi26@gmail.com,8394262988,...,4,NaN,NaN,0,5996,0.0,5996.0,6,5,284
2,T2291,2024-06-27 15:07:40,S16,HCL Mart Kolkata,Kolkata,East,C126,Anil,anil20@gmail.com,8454893002,...,-3,NaN,NaN,0,540,0.0,540.0,6,3,1114
3,T6003,2024-06-03 13:49:01,S10,HCL Mart Chandigarh,Chandigarh,North,C097,Vikram,vikram91_at_gmail.com,7346486252,...,1,PR15,Anniversary Discount,22,450,99.0,351.0,6,0,1152
4,T2423,2024-07-01 00:07:51,S20,HCL Mart Ahmedabad,Ahmedabad,West,C137,Vikram,vikram50@gmail.com,8464534706,...,-4,NaN,NaN,0,4800,0.0,4800.0,7,0,1126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,T5741,2023-01-15 12:55:13,S14,HCL Mart Bangalore,Mumbai,North,C176,Vikram,vikram51@gmail.com,9277855416,...,3,NaN,NaN,0,897,0.0,897.0,1,6,523
421,T4977,2024-09-30 00:00:00,S18,HCL Mart Mumbai,Mumbai,West,C197,Anil,anil69_at_gmail.com,7884514222,...,5,NaN,NaN,0,7500,0.0,7500.0,9,0,29
422,T1065,2023-02-12 09:01:00,S15,HCL Mart Bangalore,Bangalore,South,C084,Aman,aman33@gmail.com,7466435203,...,4,NaN,NaN,0,648,0.0,200.0,2,6,128
423,T6064,NaN,S12,HCL Mart Hyderabad,Hyderabad,South,C102,John,john91@gmail.com,7070077630,...,1,PR06,Back to School,10,1499,149.9,1349.1,6,0,132
